<a href="https://colab.research.google.com/github/andreunilux/BSP_6/blob/main/Bert_with_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#install missing dependancies
!pip install transformers accelerate


#Library used for fine tuning

from torch.utils.data import DataLoader, Dataset
from transformers import Trainer, TrainingArguments

# Pandas Dataframe Library
import json
import pandas as pd
import os
from IPython.display import display
import numpy as np 

# HuggingFace Libarary
import torch
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import AutoTokenizer, BertForSequenceClassification, AutoModelForSequenceClassification
from transformers import AutoTokenizer, BertConfig, BertTokenizer
from transformers import EarlyStoppingCallback


# Model HyperParameters
current_model="roberta-base"

#All tested models 
bert="bert-base-uncased"
HateBert="GroNLP/hateBERT"
DistilBert="distilbert-base-uncased"
RoBERTa="roberta-base" #'eval_f1': 0.5518856032046312 on gold
HateRoBERTa="facebook/roberta-hate-speech-dynabench-r4-target"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')
os.getcwd()
gold='/content/drive/MyDrive/BSP6/init_dataset_gold'
gold_silver='/content/drive/MyDrive/BSP6/init_dataset_gold_silver'
os.chdir(gold)   

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train_data = pd.read_json('train.json', lines=True)
val_data = pd.read_json('val.json', lines=True)
test_data = pd.read_json('test.json', lines=True)

In [4]:
# load pre-trained HateBert
if(current_model=="roberta-base"):
  model = AutoModelForSequenceClassification.from_pretrained('roberta-base',num_labels=3)
  tokenizer = AutoTokenizer.from_pretrained('roberta-base')
elif(current_model=="facebook/roberta-hate-speech-dynabench-r4-target"):
  tokenizer = AutoTokenizer.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target")
  model = AutoModelForSequenceClassification.from_pretrained("facebook/roberta-hate-speech-dynabench-r4-target",num_labels=3, ignore_mismatched_sizes=True,hidden_dropout_prob=0.3,attention_probs_dropout_prob=0.3)
else:
  model = BertForSequenceClassification.from_pretrained(current_model,num_labels=3 )
  tokenizer = BertTokenizer.from_pretrained(current_model)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.

In [5]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [6]:
#Train Data

X = []
for context, target in zip(train_data.context, train_data.target):
  X.append(tokenizer(context, target, padding='max_length', truncation='longest_first', max_length=230))
y_train = list(train_data.label)
print(X)
X_train_tokenized = X


#val Data
V = []
for context, target in zip(val_data.context, val_data.target):
   V.append(tokenizer(context, target, padding='max_length', truncation='longest_first', max_length=230))
y_val = list(val_data.label)
V_val_tokenized = V

#test Data
T = []
for context, target in zip(test_data.context, test_data.target):
   T.append(tokenizer(context, target, padding='max_length', truncation='longest_first', max_length=230))
y_test = list(test_data.label)
T_test_tokenized = T





IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
#print the token
print(tokenizer.decode(X_train_tokenized[0].input_ids))
print(X_train_tokenized[0])
print(len(X_train_tokenized[0].input_ids))
print(y_train[0])


print(tokenizer.decode(X_train_tokenized[1].input_ids))
print(X_train_tokenized[1])
print(len(X_train_tokenized[1].input_ids))
print(y_train[1])


<s>The UK is fucked.</s></s>>The ~~UK~~ world is fucked  FTFY</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>

In [8]:
from torch.utils.data import DataLoader, RandomSampler

# Create torch dataset
#need a dictionary
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {'input_ids':torch.tensor(self.encodings[idx].input_ids),
                'attention_mask':torch.tensor(self.encodings[idx].attention_mask),
                'labels':torch.tensor(self.labels[idx])}
        
        return item

    def __len__(self):
        return len(self.encodings)

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(V_val_tokenized, y_val)
test_dataset = Dataset(T_test_tokenized, y_test)


In [9]:
train_dataset[0]

{'input_ids': tensor([    0,   133,   987,    16, 42647,     4,     2,     2, 15698,   133,
          1437, 48256, 10494, 48256,   232,    16, 42647,  1437,   274, 20249,
           975,     2,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,   

In [10]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def compute_metrics(pred):
    labels = pred.label_ids
    #print("Predictions:"+str(pred.predictions))
    preds = pred.predictions.argmax(-1)
    accuracy = accuracy_score(labels, preds)
    print("Label"+str(labels[10:30])) 
    print("Predicted:"+str(preds[10:30]))
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}



In [11]:
# Define Trainer
args = TrainingArguments(
    output_dir='./results',           # output directory
    num_train_epochs=10,               # total number of training epochs
    per_device_train_batch_size=32,   # batch size per device during training
    per_device_eval_batch_size=32,    # batch size for evaluation
    learning_rate = 1e-5,
    evaluation_strategy="steps",
    logging_steps = 100,
    weight_decay=0.3,
    load_best_model_at_end=True
)            
                  
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    #callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
)



In [12]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
100,1.051700,1.024317,0.520337,0.495427,0.520337,0.477674
200,0.956500,0.919946,0.584853,0.561947,0.584853,0.563914
300,0.835700,0.888313,0.617111,0.599602,0.617111,0.579852
400,0.761800,0.943759,0.559607,0.585488,0.559607,0.566389
500,0.661700,0.975202,0.586255,0.587136,0.586255,0.581676
600,0.585800,1.039595,0.580645,0.581827,0.580645,0.578274
700,0.510800,1.086783,0.573633,0.573075,0.573633,0.570604
800,0.456900,1.106484,0.600281,0.597147,0.600281,0.598539
900,0.399400,1.159838,0.600281,0.596078,0.600281,0.597839
1000,0.368400,1.184795,0.579243,0.578599,0.579243,0.578663


Label[0 0 1 1 2 2 1 0 0 2 0 0 1 1 0 0 1 2 0 0]
Predicted:[1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 0 2 2 0]
Label[0 0 1 1 2 2 1 0 0 2 0 0 1 1 0 0 1 2 0 0]
Predicted:[1 1 1 1 0 2 1 0 2 2 0 1 1 1 0 1 0 2 1 0]
Label[0 0 1 1 2 2 1 0 0 2 0 0 1 1 0 0 1 2 0 0]
Predicted:[1 1 1 1 0 2 1 0 2 2 0 1 1 1 0 1 0 2 1 0]
Label[0 0 1 1 2 2 1 0 0 2 0 0 1 1 0 0 1 2 0 0]
Predicted:[0 1 1 1 0 2 1 0 2 2 2 1 1 2 2 1 0 2 2 0]
Label[0 0 1 1 2 2 1 0 0 2 0 0 1 1 0 0 1 2 0 0]
Predicted:[0 0 0 1 0 2 1 0 2 2 2 1 1 0 0 1 0 2 1 0]
Label[0 0 1 1 2 2 1 0 0 2 0 0 1 1 0 0 1 2 0 0]
Predicted:[0 0 1 1 0 2 1 0 2 2 2 1 1 0 0 1 0 2 2 0]
Label[0 0 1 1 2 2 1 0 0 2 0 0 1 1 0 0 1 2 0 0]
Predicted:[0 0 1 1 0 2 1 0 2 2 2 1 1 0 0 1 0 2 2 0]
Label[0 0 1 1 2 2 1 0 0 2 0 0 1 1 0 0 1 2 0 0]
Predicted:[1 0 1 1 0 2 1 0 2 2 2 1 1 0 0 1 0 2 2 0]
Label[0 0 1 1 2 2 1 0 0 2 0 0 1 1 0 0 1 2 0 0]
Predicted:[1 0 1 1 0 2 1 0 2 2 2 1 1 0 0 1 0 2 2 0]
Label[0 0 1 1 2 2 1 0 0 2 0 0 1 1 0 0 1 2 0 0]
Predicted:[1 0 1 1 0 2 1 0 2 2 2 1 1 0 0 1 0 2 2 0]


TrainOutput(global_step=1040, training_loss=0.6477424749961266, metrics={'train_runtime': 1288.1743, 'train_samples_per_second': 25.812, 'train_steps_per_second': 0.807, 'total_flos': 3929999730615000.0, 'train_loss': 0.6477424749961266, 'epoch': 10.0})

In [13]:
eval=trainer.evaluate(val_dataset)

Label[0 0 1 1 2 2 1 0 0 2 0 0 1 1 0 0 1 2 0 0]
Predicted:[0 0 0 1 0 2 1 0 2 2 2 1 1 0 0 1 0 2 1 0]


In [14]:
eval


{'eval_loss': 0.9752022624015808,
 'eval_accuracy': 0.5862552594670407,
 'eval_precision': 0.5871358163881257,
 'eval_recall': 0.5862552594670407,
 'eval_f1': 0.581676267823053,
 'eval_runtime': 8.5625,
 'eval_samples_per_second': 83.271,
 'eval_steps_per_second': 2.686,
 'epoch': 10.0}